## Computational ressources

In [8]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 

cluster.scale(204) 

from dask.distributed import Client
client = Client(cluster)

client

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://172.30.100.4:46735 Dashboard: http://172.30.100.4:38072/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [9]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           7479974      allc make_pro albert7a  R       2:12      1 n2418
           7479975      allc make_pro albert7a  R       2:12      1 n2419
           7479976      allc make_pro albert7a  R       2:12      1 n2420
           7479977      allc make_pro albert7a  R       2:12      1 n2421
           7479978      allc make_pro albert7a  R       2:12      1 n2422
           7479979      allc make_pro albert7a  R       2:12      1 n2423
           7479972      allc make_pro albert7a  R       2:15      1 n2362
           7479973      allc make_pro albert7a  R       2:15      1 n2363


## Librairies and customization

In [3]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
today=date.today()

import sys
sys.path.insert(0,'/home/albert7a/git/xscale')
import xscale

import xscale.spectral.fft as xfft
from xscale.spectral.tools import plot_spectrum
import xscale.signal.generator as xgen

import matplotlib.pyplot as plt
params = {'figure.figsize' : (15, 8),'legend.fontsize': 16,'xtick.labelsize':16,'ytick.labelsize':16,'axes.labelsize':16,'font.size':16}
plt.rcParams.update(params)


In [10]:
tfilenames_FMA = []

for month in ['02','03','04']:
    if month in ['12','01','02','03','04','05','06']:
        case='BLBT02X'
    else:
        case='BLBT02'

    if month in ['07','08','09','10','11','12']:
        year='2009'
    else:
        year='2010'
    
    data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-'+str(case)+'-S/'
    tfilenames_month = sorted(glob.glob(data_dir+'*/eNATL60-'+str(case)+'_1h_*_gridT-2D_'+str(year)+str(month)+'??-'+str(year)+str(month)+'??.nc'))
    tfilenames_FMA=tfilenames_FMA+tfilenames_month

In [11]:
%time dsFMA=xr.open_mfdataset(tfilenames_FMA,chunks={'x':1000,'y':1000,'time_counter':1})['sossheig']


CPU times: user 1min 35s, sys: 50.6 s, total: 2min 25s
Wall time: 3min 39s


In [10]:
dsFMA_reg=dsFMA[:,1750:2250,3750:4250].chunk({'x':250,'y':250,'time_counter':100})

In [11]:
NPADDING = 2
%time SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)


distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

CPU times: user 53 s, sys: 17.4 s, total: 1min 10s
Wall time: 1min 29s


In [12]:
%time MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()


KilledWorker: ('xarray-<this-array>-e35c49e7d7f2cbf6205c626d65cc077d', <Worker 'tcp://172.30.5.254:44918', memory: 0, processing: 1>)

In [13]:
dsFMA_reg=dsFMA[:,1750:2250,3750:4250].chunk({'x':250,'y':250,'time_counter':10})
NPADDING = 2
SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)
MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()


distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

KilledWorker: ('xarray-<this-array>-945db281319533744ad0572af38f8921', <Worker 'tcp://172.30.5.251:33802', memory: 0, processing: 1>)

In [14]:
dsFMA_reg=dsFMA[:,1750:2250,3750:4250].chunk({'x':250,'y':250,'time_counter':1})
NPADDING = 2
SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)
MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()


distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 18% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

KilledWorker: ('xarray-<this-array>-e014d5f692787e73880f89d1f5916757', <Worker 'tcp://172.30.5.253:41919', memory: 0, processing: 1>)

In [15]:
dsFMA_reg=dsFMA[:,1750:2250,3750:4250].chunk({'x':500,'y':500,'time_counter':1})
NPADDING = 2
SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)
MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()


distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 17% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

KilledWorker: ('xarray-<this-array>-a4c390c7d564a8e289affe4053c40308', <Worker 'tcp://172.30.5.253:37150', memory: 0, processing: 1>)

distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)


In [6]:
dsFMA_reg=dsFMA[:,1750:2250,3750:4250].chunk({'x':500,'y':500,'time_counter':10})
NPADDING = 2
SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)
MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()


KilledWorker: ('xarray-<this-array>-84f293fc4e7645283ed2d3ea7d06c788', <Worker 'tcp://172.30.10.84:45225', memory: 0, processing: 1>)

In [13]:
dsFMA_reg=dsFMA[:,1750:2250,3750:4250].chunk({'x':500,'y':500,'time_counter':100})
NPADDING = 2
SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)
MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


OSError: [Errno -101] NetCDF: HDF error: b'/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02X-S/00950401-01004400/eNATL60-BLBT02X_1h_20100317_20100410_gridT-2D_20100406-20100406.nc'

In [8]:
dsFMA_reg=dsFMA[:,1750:2250,3750:4250].chunk({'x':500,'y':500,'time_counter':1000})
NPADDING = 2
SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)
MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()


distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

OSError: [Errno -101] NetCDF: HDF error: b'/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02X-S/00842401-00864000/eNATL60-BLBT02X_1h_20100126_20100204_gridT-2D_20100201-20100201.nc'

distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)


In [6]:
dsFMA_reg=dsFMA[:,1750:2250,3750:4250].chunk({'x':250,'y':250,'time_counter':1000})
NPADDING = 2
SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)
MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()


KilledWorker: ('xarray-<this-array>-0cca038066676ac3294a7efeba6867a3', <Worker 'tcp://172.30.8.81:36353', memory: 0, processing: 1>)

In [7]:
dsFMA_reg=dsFMA[:,1750:2250,3750:4250].chunk({'x':100,'y':100,'time_counter':2160})
NPADDING = 2
SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)
MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()


KilledWorker: ('xarray-<this-array>-0333512416c3334d769ea83028cb685b', <Worker 'tcp://172.30.10.84:38344', memory: 0, processing: 1>)

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


In [12]:
dsFMA_reg=dsFMA[:,1750:2250,3750:4250]
NPADDING = 2
SPtime_FMA_reg = xfft.fft(dsFMA_reg, dim='time_counter', dx=1., detrend='mean',nfft=dsFMA_reg.shape[0]*NPADDING,tapering=True)
MEANPSD_FMA=xfft.psd(SPtime_FMA_reg).mean(dim='x').mean(dim='y').load()


KilledWorker: ('xarray-<this-array>-ec669a7e3c4e62cf50f4d167522dca64', <Worker 'tcp://172.30.6.192:43760', memory: 0, processing: 1>)

In [13]:
cluster.close()

In [14]:
client.close()

Future exception was never retrieved
future: <Future finished exception=OSError("Timed out trying to connect to 'tcp://172.30.6.188:45504' after 10 s: in <distributed.comm.tcp.TCPConnector object at 0x2ac6186f52b0>: ConnectionRefusedError: [Errno 111] Connection refused")>
Traceback (most recent call last):
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/comm/tcp.py", line 359, in connect
    ip, port, max_buffer_size=MAX_BUFFER_SIZE, **kwargs
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 729, in run
    value = future.result()
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/tornado/tcpclient.py", line 280, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
  File "/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/tornado/tcpclient.py", line 143, in on_connect_done
    stream = future.result()
tornado.ios